# Import Statements

In [18]:
import OCR_pipeline as OCR
import nltk
import spacy


# Text Extraction

In [19]:
textDictionary = OCR.extractTextFromAllPapers("Kenya")
print(textDictionary)

#k = OCR.extractTextFromPaper("kenya/AgricultureFisheriesandFoodAuthorityNo13of2013.PDF")
#print(k)

# Sentence Conversion Method

In [26]:
#? does bigram_mod and trigram_mod need to be inside method?
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#returns list of cleaned sentences
def toSentences(pageList):
    
    #convert into long string (from list of page texts)
    longString = ""
    for sent in pageList:
        longString += sent
    
    #split into list of sentences
    sentences = nltk.sent_tokenize(longString)
        
    # Remove Stop Words
    sentences_nostops = remove_stopwords(sentences_words)
    
    # Form Bigrams
    sentences_bigrams = make_bigrams(sentences_nostops)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])
    sentences_lemmatized = lemmatization(sentences_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    return sent_tokenize(sentences_lemmatized)
        

# Cleaning

In [23]:
#convert each list of pages into one of cleaned sentences
textDictionary = {}
for key in textDictionary:
    currentText = textDictionary[key]
    textDictionary[key] = toSentences(textDictionary[key])
    

NameError: name 'textDictionary' is not defined

# Topic Model Method

In [25]:
#runs LDA on lemmatized text and returns corpus and LDA model
def LDA(lemmatized_sents):
    #create dictionary
    id2word = corpora.Dictionary(lemmatized_sents)
    
    # Creates corpus
    corpus = [id2word.doc2bow(lemmatized_sents) for lemmatized_sents in lemmatized_sents]
    
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    return [corpus, lda_model]
    


In [ ]:
# Print the Keywords in the 5 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]